## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb　←いまここ
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

## ここから本編

In [ ]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import re

研究課題の主な項目をデータフレームにして、pickleして保存する関数を定義する。

In [ ]:
def kadai(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    kadailist = []
    for grantAward in tree.iterfind("grantAward"):
        projecttype = grantAward.get("projectType")
        awardnumber = grantAward.get("awardNumber")
        startfiscalyear = grantAward.find("summary[@xml:lang='ja']/periodOfAward", nsmap).get("searchStartFiscalYear")
        endfiscalyear = grantAward.find("summary[@xml:lang='ja']/periodOfAward", nsmap).get("searchEndFiscalYear")

        try:
            category_niicode = grantAward.find("summary[@xml:lang='ja']/category", nsmap).get("niiCode")
        except:
            category_niicode = np.NaN
        
        try:
            category = grantAward.find("summary[@xml:lang='ja']/category", nsmap).text
        except:
            category = np.NaN
            
        try:
            section_niicode = grantAward.find("summary[@xml:lang='ja']/section", nsmap).get("niiCode")
        except:
            section_niicode = np.NaN
        
        try:
            section = grantAward.find("summary[@xml:lang='ja']/section", nsmap).text
        except:
            section = np.NaN

        try:
            title_ja = grantAward.find("summary[@xml:lang='ja']/title", nsmap).text
        except:
            title_ja = np.NaN
        
        try:
            title_en = grantAward.find("summary[@xml:lang='en']/title", nsmap).text
        except:
            title_en = np.NaN
        
        try:
            directcost = grantAward.find("summary[@xml:lang='ja']/overallAwardAmount/directCost", nsmap).text
        except:
            directcost = np.NaN

        row = [
            awardnumber,
            projecttype,
            category,
            category_niicode,
            section,
            section_niicode,
            startfiscalyear,
            endfiscalyear,
            directcost,
            title_ja,
            title_en,
        ]
        kadailist.append(row)
        
    df = pd.DataFrame(kadailist)
    df.columns = [
        'awardnumber',
        'projecttype',
        'category',
        'category_niicode',
        'section',
        'section_niicode',
        'startfiscalyear',
        'endfiscalyear',
        'directcost',
        'title_ja',
        'title_en',
    ]
    
    pickledfile = 'pickledDF_grantaward_main/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
    df.to_pickle(pickledfile)

フォルダをいったんきれいにする関数を定義して、実行する

In [ ]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_grantaward_main')

2010年以降のxmlファイルをパースして、1個ずつpickleして保存

In [ ]:
for xmlfile in tqdm(glob.glob('xml/201*.xml')):
    kadai(xmlfile)

空のデータフレームを作って…

In [ ]:
columns = [
    'awardnumber',
    'projecttype',
    'category',
    'category_niicode',
    'section',
    'section_niicode',
    'startfiscalyear',
    'endfiscalyear',
    'directcost',
    'title_ja',
    'title_en',
]
df = pd.DataFrame(columns=columns)

pickleからデータフレームを復元、すべて結合

In [ ]:
for dump in tqdm(glob.glob('pickledDF_grantaward_main/201*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

連結したデータフレームをいったんpickleして保存

In [ ]:
import os
os.makedirs('beforeCleaning', exist_ok=True)

df.to_pickle('beforeCleaning/parse_grantaward_main.dump')

### データクリーニング

pickleからデータフレームを復元する

In [ ]:
with open('beforeCleaning/parse_grantaward_main.dump', mode='rb') as f:
    df = pickle.load(f)

データを概観する

In [ ]:
df

重複したデータがあるかどうか

In [ ]:
df.duplicated().any()

重複はなかった（あったら何かが違う）。

awardnumberがユニークかどうか確認するために、行数255813とawardnumberのユニークな値の数が一致するかどうか確認する。

In [ ]:
df['awardnumber'].nunique(dropna=False)

awardnumberはユニークだったので（ユニークじゃなかったら何かが違う）、インデックスにする。

In [ ]:
df = df.set_index('awardnumber')
df

構造を見ておく

In [ ]:
df.info()

nullを0に置換しておく。

In [ ]:
df = df.fillna(0)
df.info()

データ型を指定しておく。

In [ ]:
df = df.astype({
    'category_niicode': np.int64,
    'section_niicode': np.int64,
    'startfiscalyear': np.int64,
    'endfiscalyear': np.int64,
    'directcost': np.int64,
})
df.info()

directcostを円単位から、千円単位にする。

In [ ]:
df['directcost'] = df['directcost'].map(lambda x: x // 1000)

niicodeを精査する

In [ ]:
pd.crosstab(df['category_niicode'], df['startfiscalyear'], margins=True)

niicodeが0になっているものが9件あった。まず9件のデータを見る。

In [ ]:
df[df['category_niicode'] == 0]

タイトルなどをgoogle検索したところ、種目が判明した。category_niicodeを代入しておく。

In [ ]:
df.loc['22128009', 'category_niicode'] = 73 #  新学術領域研究(研究領域提案型)
df.loc['22900002', 'category_niicode'] = 55 #  特別研究推進費
df.loc['22900001', 'category_niicode'] = 55 #  特別研究推進費
df.loc['11F01767', 'category_niicode'] = 42 #  特別研究員奨励費
df.loc['11F01514', 'category_niicode'] = 42 #  特別研究員奨励費
df.loc['11F01303', 'category_niicode'] = 42 #  特別研究員奨励費
df.loc['23900002', 'category_niicode'] = 55 #  特別研究推進費
df.loc['23900001', 'category_niicode'] = 55 #  特別研究推進費
df.loc['24900001', 'category_niicode'] = 55 #  特別研究推進費

category_niicodeが全部決まったことを確認する。

In [ ]:
pd.crosstab(df['category_niicode'], df['startfiscalyear'], margins=True)

クリーニング後の課題データフレームをpickleして保存する

In [ ]:
import os
os.makedirs('afterCleaning', exist_ok=True)

df.to_pickle('afterCleaning/parse_kadai.dump')

ひとまず終了。

次は、parse_institution.ipynbで内定時の研究機関を取得する。